AI Ranger Team Demo Development
# Deep Learning for Medical Image Analysis leveraging the AML Platform
### Pneunomia detection using Remote Experiment Runs and Azure ML Pipeines

<img src="images/medicalimage.jpg" width=1000 />

### In this notebook
In this notebook, three different approaches are demonstrated to train a Pneumonia detection model using the unique capabilities of Azure ML. We will start by training a baseline model, which is trained on a remote cluster with GPU machines. The second part will show an approach for training hyperparameters. The third and last part of the notebook, will transform the Remote Experiment Runs in a Azure ML Pipeline, that can be used to train a model repeatably when new data is available. The Pipeline will also include a step for deployment.

# Pneumonia Detection Use Case
A relatively small public dataset of medical images for detecting viral or bacterial pneumonia has been chosen to keep the scenario straightforward and reproducible with limited computing resources. The dataset contains 5,218 x-ray images with two classes of diagnostic outcomes: 3,876 cases with (viral or bacterial) pneumonia and 1,342 cases without findings ("Normal").
The dataset is split into training, validation and test sets. Since some images represent radiographs from the same patient, it has been ensured that there is no overlap of patients between the training, validation and test sets.

<img src="images/pneumonia.png" width=1000 />
You can find the dataset under this location: https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia. 


# Neural Network architecture

The following neural network architecture is used:

<img src="images/cnnframe.png" width=1200 />

Though a detailed discussion of the architecture and functionality of Convnets is outside the scope of this demo, the following summary provides a brief overview ofthe design:
- The x-ray images are resized to a 224 x 224 pixel resolution before being fed into the Convnet. Other medical imaging use cases will most likely require higher resolutions. However, for the selected dataset, high accuracy results can be achieved with this small image size.
- During  the  data  flow  through  the Convnet, relevant  properties  for  the  classification  task (features) are extracted in a hierarchical way. The lower layers of the network detect low-level features like edges or surfaces. More complex features (for detecting pneumonia in this case) are extracted at higher layers. The three convolutional layers perform the detection of features at different abstraction levels in the network, where the images are scanned by a small moving window (kernel).
- To reduce computational effort while focusing on the most dominant features, the image size is reduced further as the data flows through the three max pooling layers.
- Two dropout layers are included to reduce the risk of overfitting to the training data.
- The final layer consists of two neurons for representing the classes "pneumonia" and "normal".

# Setup


## Installs and imports

In [1]:
!pip install split-folders

/bin/bash: pip: command not found


In [2]:
# Download Kaggle pip package and split-folders
%pip install kaggle --upgrade split-folders

Requirement already up-to-date: kaggle in /anaconda/envs/azureml_py38/lib/python3.8/site-packages (1.5.12)
Requirement already up-to-date: split-folders in /anaconda/envs/azureml_py38/lib/python3.8/site-packages (0.5.1)
Note: you may need to restart the kernel to use updated packages.


In [5]:
import sys
!{sys.executable} -m pip install kaggle
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia -p /tmp

100%|█████████████████████████████████████▉| 2.29G/2.29G [00:26<00:00, 72.5MB/s]
100%|██████████████████████████████████████| 2.29G/2.29G [00:26<00:00, 93.0MB/s]


In [1]:
import matplotlib.pyplot as plt

import json
from azureml.core import Workspace, Dataset, Experiment

workspace = Workspace.from_config()

## Retrieve and upload data

When you use this notebook for the first time, the pneumonia dataset should be uploaded to the default AzureML datastore and registered as a managed file dataset.

The commands below can be used to download the dataset using the Kaggle API (https://github.com/Kaggle/kaggle-api). Use the instructions to generate your own API key and fill them in on the code cell.

In [2]:
# Export Kaggle configuration variables
%env KAGGLE_USERNAME=[Kaggle user name]
%env KAGGLE_KEY=[API token]

env: KAGGLE_USERNAME=[Kaggle user name]
env: KAGGLE_KEY=[API token]


In [1]:
# Export Kaggle configuration variables
%env KAGGLE_USERNAME=ankopp
%env KAGGLE_KEY=7400527eb41ba5d3248625bea6f1a3bb

env: KAGGLE_USERNAME=ankopp
env: KAGGLE_KEY=7400527eb41ba5d3248625bea6f1a3bb


In [3]:
# remove folders and zipfile from previous runs of the cell
!rm /tmp/chest-xray-pneumonia.zip
!rm -r /tmp/chest_xray
!rm -r /tmp/chest_xray_tvt

# Download the Pneumonia dataset
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia -p /tmp

!unzip -q /tmp/chest-xray-pneumonia.zip -d /tmp

rm: cannot remove '/tmp/chest-xray-pneumonia.zip': No such file or directory
rm: cannot remove '/tmp/chest_xray': No such file or directory
rm: cannot remove '/tmp/chest_xray_tvt': No such file or directory
100%|██████████████████████████████████████▉| 2.29G/2.29G [00:34<00:00, 110MB/s]
100%|██████████████████████████████████████| 2.29G/2.29G [00:34<00:00, 72.1MB/s]


In [4]:
import splitfolders

download_root = '/tmp/chest_xray/train' 
train_val_test_root = '/tmp/chest_xray_tvt/'

train_val_test_split = (0.8, 0.1, 0.1)
random_seed = 33

splitfolders.ratio(download_root, train_val_test_root, random_seed, ratio=train_val_test_split)

Copying files: 5216 files [00:01, 2951.35 files/s]


In [5]:
# check dataset splits
for split in os.listdir(train_val_test_root):
    for label in ['NORMAL', 'PNEUMONIA']:
        files = os.listdir(os.path.join(train_val_test_root, split, label))
        print(f'{split}-{label}: ', len(files))


val-NORMAL:  134
val-PNEUMONIA:  387
train-NORMAL:  1072
train-PNEUMONIA:  3100
test-NORMAL:  135
test-PNEUMONIA:  388


In [6]:
from azureml.core import Workspace, Datastore, Dataset
from azureml.data.datapath import DataPath

# Upload data to AzureML Datastore
ds = workspace.get_default_datastore()
ds = Dataset.File.upload_directory(src_dir=train_val_test_root,
            target=DataPath(ds, 'chest-xray'),
            show_progress=False, overwrite=False)

# Register file dataset with AzureML
ds = ds.register(workspace=workspace, name="pneumonia", description="Pneumonia train / val / test folders with 2 classes", create_new_version=True)

print(f'Dataset {ds.name} registered.')

Dataset pneumonia registered.


# I. Run baseline experiment

## Create/retrieve Compute Cluster

In [7]:
from azureml.core.compute import AmlCompute, ComputeTarget

cluster_name = "gpu-cluster"

try:
    compute_target = workspace.compute_targets[cluster_name]
    print('Found existing compute target.')
except KeyError:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_NC6', 
                                                           idle_seconds_before_scaledown=1800,
                                                           min_nodes=0, 
                                                           max_nodes=4)

    compute_target = ComputeTarget.create(workspace, cluster_name, compute_config)
    
# Can poll for a minimum number of nodes and for a specific timeout.
# If no min_node_count is provided, it will use the scale settings for the cluster.
compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

Found existing compute target.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


### Define ScriptRunConfig object

Define the Environment that you will use to run your experiment, retrieve the dataset by name and define the ScriptRunConfig object.

In [8]:
from azureml.core import ScriptRunConfig, Environment
from azureml.core.compute import ComputeTarget

experiment = Experiment(workspace, 'pneumonia')

pytorch_env = Environment.from_conda_specification(name = 'pytorch-1.6-gpu', file_path = './training/conda_dependencies.yml')

dataset = Dataset.get_by_name(workspace, name='pneumonia', version='latest')

src = ScriptRunConfig(source_directory='./training',
                      script='train.py',
                      arguments=['--epochs', 15, '--data-folder', dataset.as_mount()],
                      compute_target= ComputeTarget(workspace, 'gpu-cluster'),
                      environment=pytorch_env)

## Submit baseline experiment

In [9]:
from azureml.widgets import RunDetails

script_run = experiment.submit(src)
RunDetails(script_run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [ ]:
script_run.wait_for_completion()

# II. Hyperparameter tuning using Random Parameter Sampling
Hyperparameter tuning, also called hyperparameter optimization, is the process of finding the configuration of hyperparameters that results in the best performance. The process is typically computationally expensive and manual.

Azure Machine Learning lets you automate hyperparameter tuning and run experiments in parallel to efficiently optimize hyperparameters.

Random sampling supports discrete and continuous hyperparameters. It supports early termination of low-performance runs. Some users do an initial search with random sampling and then refine the search space to improve results. In random sampling, hyperparameter values are randomly selected from the defined search space.

Selected hyperparameters affect various stages of the experiment:

- Data: Training and validation loader: batch size
- CNN Architecture: Dropout
- Choice of optimizer
- Training loop: learning rate

In [11]:
from azureml.train.hyperdrive import RandomParameterSampling, BanditPolicy, HyperDriveConfig, uniform, choice, PrimaryMetricGoal

param_sampling = RandomParameterSampling( {
        'learning_rate': choice(0.00007, 0.0007, 0.07),
        'batch_size': choice(16, 32, 64, 128), 
        'conv_dropout' : uniform(0.0, 0.5), 
        'optimizer': choice('SGD', 'Adam', 'RMSprop')
    }
)

early_termination_policy = BanditPolicy(slack_factor=0.15, evaluation_interval=1, delay_evaluation=5)

hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=param_sampling, 
                                     policy=early_termination_policy,
                                     primary_metric_name='best_val_acc',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=8,
                                     max_concurrent_runs=4)

## Submit hyperdrive run

In [12]:
from azureml.widgets import RunDetails

# start the HyperDrive run
hyperdrive_run = experiment.submit(hyperdrive_config)

RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [15]:
hyperdrive_run.wait_for_completion()

{'runId': 'HD_b4b229ae-bff3-428c-9485-4e0fbe0f984f',
 'target': 'gpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-04-12T18:28:01.345209Z',
 'endTimeUtc': '2022-04-12T19:21:43.355893Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "best_val_acc", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '7c0db779-872d-44e5-829f-16a13b13aa85',
  'user_agent': 'python/3.8.5 (Linux-5.4.0-1073-azure-x86_64-with-glibc2.10) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.39.0',
  'space_size': 'infinite_space_size',
  'score': '0.98828125',
  'best_child_run_id': 'HD_b4b229ae-bff3-428c-9485-4e0fbe0f984f_4',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://livecell0183157347.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_b4b229ae-bff3-428c-9485-4e0fbe0f984

#  III. Define AML pipeline with HyperDriveStep

The third part of this, is a showcase on how to use the Azure ML Pipeline capability to create a pipeline from the same training script that we have been using. In the cell below, the first step of the pipeline is created, by defining the pipeline data that will be the output of the first step.
The Hyperdrive config that we have defined in the previous step will be re-used.

In [17]:
from azureml.pipeline.steps import HyperDriveStep, HyperDriveStepRun, PythonScriptStep
from azureml.pipeline.core import Pipeline, PipelineData, TrainingOutput

metrics_output_name = 'metrics_output'
metrics_data = PipelineData(name='metrics_data',
                            datastore=workspace.get_default_datastore(),
                            pipeline_output_name=metrics_output_name,
                            training_output=TrainingOutput("Metrics"))

model_output_name = 'model_output'
saved_model = PipelineData(name='saved_model',
                            datastore=workspace.get_default_datastore(),
                            pipeline_output_name=model_output_name,
                            training_output=TrainingOutput("Model",
                                                           model_file="outputs/model/pneumonia.pt"))

hd_step_name='hyperdrive_step'
hd_step = HyperDriveStep(
    name=hd_step_name,
    hyperdrive_config=hyperdrive_config,
    inputs=[dataset.as_mount()],
    outputs=[metrics_data, saved_model])


## Find and register best model

We add a step in our pipeline to find and register the best model, that is the output of the Hyperdrivestep.

In [18]:
%%writefile training/register_model.py

import argparse
import json
import os
from azureml.core import Workspace, Experiment, Model
from azureml.core import Run
from shutil import copy2

parser = argparse.ArgumentParser()
parser.add_argument('--saved-model', type=str, dest='saved_model', help='path to saved model file')
args = parser.parse_args()

model_output_dir = './model/'

os.makedirs(model_output_dir, exist_ok=True)
copy2(args.saved_model, model_output_dir)

ws = Run.get_context().experiment.workspace

model = Model.register(workspace=ws, model_name='pneunomia-pytorch', model_path=model_output_dir)

Overwriting training/register_model.py


In [19]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies

conda_dep = CondaDependencies()
conda_dep.add_pip_package("azureml-sdk")

rcfg = RunConfiguration(conda_dependencies=conda_dep)

register_model_step = PythonScriptStep(source_directory='./training',
                                       script_name='register_model.py',
                                       name="register_model_step01",
                                       inputs=[saved_model],
                                       compute_target=ComputeTarget(workspace, 'gpu-cluster'),
                                       arguments=["--saved-model", saved_model],
                                       allow_reuse=True,
                                       runconfig=rcfg)

register_model_step.run_after(hd_step)

## Submit pipeline including model registration

In [20]:
pipeline = Pipeline(workspace=workspace, steps=[hd_step, register_model_step])
pipeline_run = experiment.submit(pipeline)

Created step hyperdrive_step [5ffc0cdb][aad1a671-435e-437c-9cd5-38ef6104bfb0], (This step will run and generate new outputs)
Created step register_model_step01 [2c2591f9][9d2e8a1e-8a54-414c-8c01-cb88f9eb61c4], (This step will run and generate new outputs)
Submitted PipelineRun 81f7f0b9-8264-4964-9afc-b0ede5edabd7
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/81f7f0b9-8264-4964-9afc-b0ede5edabd7?wsid=/subscriptions/4eeedd72-d937-4243-86d1-c3982a84d924/resourcegroups/livecell/workspaces/livecell&tid=72f988bf-86f1-41af-91ab-2d7cd011db47


In [22]:
pipeline_run.wait_for_completion()

PipelineRunId: 81f7f0b9-8264-4964-9afc-b0ede5edabd7
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/81f7f0b9-8264-4964-9afc-b0ede5edabd7?wsid=/subscriptions/4eeedd72-d937-4243-86d1-c3982a84d924/resourcegroups/livecell/workspaces/livecell&tid=72f988bf-86f1-41af-91ab-2d7cd011db47
PipelineRun Status: Running


StepRunId: 5021ddd2-7149-4848-9fe2-8393e06b083e
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/5021ddd2-7149-4848-9fe2-8393e06b083e?wsid=/subscriptions/4eeedd72-d937-4243-86d1-c3982a84d924/resourcegroups/livecell/workspaces/livecell&tid=72f988bf-86f1-41af-91ab-2d7cd011db47
StepRun( hyperdrive_step ) Status: Running

StepRun(hyperdrive_step) Execution Summary
StepRun( hyperdrive_step ) Status: Finished
{'runId': '5021ddd2-7149-4848-9fe2-8393e06b083e', 'status': 'Completed', 'startTimeUtc': '2022-04-12T19:24:39.086161Z', 'endTimeUtc': '2022-04-12T20:15:52.625381Z', 'services': {}, 'properties': {'ContentSnapshotId': '2ba0b739-1103-42ef-b04c-a29dcb7f2

'Finished'

## Download training metrics

In [23]:
metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)

Downloaded azureml/5021ddd2-7149-4848-9fe2-8393e06b083e/metrics_data, 1 files out of an estimated total of 1


## Visualize training metrics

In [24]:
import pandas as pd
import json
with open(metrics_output._path_on_datastore) as f:  
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

,HD_71908756-cfd7-4d68-93ca-e43576536b69_2,HD_71908756-cfd7-4d68-93ca-e43576536b69_1,HD_71908756-cfd7-4d68-93ca-e43576536b69_4,HD_71908756-cfd7-4d68-93ca-e43576536b69_3,HD_71908756-cfd7-4d68-93ca-e43576536b69_5,HD_71908756-cfd7-4d68-93ca-e43576536b69_7,HD_71908756-cfd7-4d68-93ca-e43576536b69_6,HD_71908756-cfd7-4d68-93ca-e43576536b69_0
validation accuracy,"[0.73828125, 0.73828125, 0.73828125, 0.7382812...","[0.974609375, 0.96484375, 0.970703125, 0.98046...","[0.84375, 0.84375, 0.82421875, 0.86328125, 0.8...","[0.73828125, 0.73828125, 0.85546875, 0.8945312...","[0.939453125, 0.95703125, 0.962890625, 0.96484...","[0.73828125, 0.73828125, 0.91796875, 0.9375, 0...","[0.73828125, 0.73828125, 0.73828125, 0.7382812...","[0.73828125, 0.73828125, 0.73828125, 0.7382812..."
training accuracy,"[0.7221153846153846, 0.7324519230769231, 0.743...","[0.9185823754789272, 0.9674329501915708, 0.976...","[0.7305288461538462, 0.8079326923076923, 0.824...","[0.7326923076923076, 0.7317307692307692, 0.755...","[0.7466346153846154, 0.95, 0.9572115384615385,...","[0.7275390625, 0.742431640625, 0.8486328125, 0...","[0.643310546875, 0.742431640625, 0.744140625, ...","[0.740234375, 0.742431640625, 0.744140625, 0.7..."
validation loss,"[0.5886029446651291, 0.5719553907178414, 0.568...","[0.06982662478701396, 0.08999283778375719, 0.0...","[16.72988772023319, 11.629959796615378, 6.9222...","[0.40198286938804584, 0.49974885233990746, 0.3...","[0.164949843155903, 0.11227239665509185, 0.101...","[0.6033398706945028, 0.3253373440762628, 0.289...","[0.5729615546462632, 0.5667705572498051, 0.564...","[0.5934913620610155, 0.5801316845211095, 0.574..."
training loss,"[3781.522757672511, 0.5814548206922256, 0.5730...","[0.21702057078822784, 0.08767054384660701, 0.0...","[183445394.75855544, 18.088725348171124, 36.93...","[0.5625044819500274, 0.5000821459755151, 0.469...","[0.6216836162641937, 0.13374727813322998, 0.10...","[12.87439876831872, 0.3617945369609174, 0.3035...","[52113.18643739975, 0.5630050214040456, 0.5574...","[0.5990463961031607, 0.564624817390752, 0.5563..."
best_val_acc,"[0.73828125, 0.73828125, 0.73828125, 0.7382812...","[0.974609375, 0.974609375, 0.974609375, 0.9804...","[0.84375, 0.84375, 0.84375, 0.86328125, 0.8632...","[0.73828125, 0.73828125, 0.85546875, 0.8945312...","[0.939453125, 0.95703125, 0.962890625, 0.96484...","[0.73828125, 0.73828125, 0.91796875, 0.9375, 0...","[0.73828125, 0.73828125, 0.73828125, 0.7382812...","[0.73828125, 0.73828125, 0.73828125, 0.7382812..."
Train imgs,[4182.0],[4182.0],[4182.0],[4182.0],[4182.0],[4182.0],[4182.0],[4182.0]


## Publish the training pipeline

By publishing the training pipeline, an pipeline endpoint is created, that we can use to trigger the pipeline from external services.

In [25]:
published_pipeline1 = pipeline_run.publish_pipeline(
     name="Training_pneumonia",
     description="Pipeline to train a classification model to detect pneumonia.",
     version="1.0")

print(published_pipeline1.id)

a93bf4c1-2fba-4def-b7ee-c5d89e06567a


## Create a schedule based on file change
One advantage of defining and publishing your training script as an Azure ML Pipeline, is that a schedule can be created to trigger retraining of your model based on file changes in the source dataset.

In [2]:
from azureml.pipeline.core.schedule import Schedule

schedule = Schedule.list(workspace) 
print(schedule)

# sch = Schedule.list(workspace)[0] # I want to disable the first pipeline
    
# Schedule.disable(sch)

[Pipeline(Name: MyReactiveSchedule,
Id: f95649d0-1a91-410a-a647-da823538be60,
Status: Active,
Pipeline Id: f2ad64dc-ef8a-4f3d-b199-35c14c8d8b97,
Pipeline Endpoint Id: None,
Datastore: workspaceblobstore,
Path on Datastore: chest-xray/train/PNEUMONIA)]


In [18]:
from azureml.pipeline.core import PipelineEndpoint

PipelineEndpoint.list(workspace)

# PipelineEndpoint.get(workspace=workspace, name="Training_pneumonia")

[]

In [26]:
from azureml.pipeline.core.schedule import Schedule
from azureml.pipeline.core import PipelineEndpoint

datastore = workspace.get_default_datastore()

# pipeline_endpoint_by_name = PipelineEndpoint.get(workspace=workspace, name="Training_pneumonia")

reactive_schedule = Schedule.create(workspace, name="MyReactiveSchedule", 
                                    description="Based on input file change.",
                                    pipeline_id=published_pipeline1.id,
                                    experiment_name='experiment_name',
                                    datastore=datastore, 
                                    path_on_datastore="chest-xray/train/PNEUMONIA")
                                    #data_path_parameter_name="input_data")


In [27]:
from azureml.data.datapath import DataPath

samples = './chest-xray-retrain'

ds = workspace.get_default_datastore()
ds = Dataset.File.upload_directory(src_dir = samples,
            target=DataPath(ds, 'chest-xray/train'),
            show_progress=True, overwrite=False)


Validating arguments.
Arguments validated.
Uploading file to chest-xray/train
Uploading an estimated of 10 files
Target already exists. Skipping upload for chest-xray/train/NORMAL/add_IM-0077-0001.jpeg
Target already exists. Skipping upload for chest-xray/train/NORMAL/add_NORMAL2-IM-0146-0001.jpeg
Target already exists. Skipping upload for chest-xray/train/NORMAL/add_NORMAL2-IM-0198-0001.jpeg
Target already exists. Skipping upload for chest-xray/train/NORMAL/add_NORMAL2-IM-0241-0001.jpeg
Target already exists. Skipping upload for chest-xray/train/NORMAL/add_NORMAL2-IM-0276-0001.jpeg
Target already exists. Skipping upload for chest-xray/train/PNEUMONIA/add_person133_bacteria_637.jpeg
Target already exists. Skipping upload for chest-xray/train/PNEUMONIA/add_person1628_virus_2821.jpeg
Target already exists. Skipping upload for chest-xray/train/PNEUMONIA/add_person1676_virus_2892.jpeg
Target already exists. Skipping upload for chest-xray/train/PNEUMONIA/add_person1_virus_13.jpeg
Target alr